# Blackbody Radiation

This notebook contains the programmatic verification for the **Blackbody Radiation** entry from the THEORIA dataset.

**Entry ID:** blackbody_radiation  
**Required Library:** sympy 1.13.1

## Description
Planck's law describes the spectral energy density of black-body radiation as a function of frequency and temperature. By quantizing electromagnetic modes with energy quanta `h*nu`, it resolves the ultraviolet catastrophe of classical physics and underpins modern quantum theory and thermal emission models.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.13.1

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# ===============================
# Programmatic verification: Blackbody Radiation (Planck law)
# Robust to SymPy summation quirks and includes classic checks.
# ===============================

# Symbols
nu, T, h, k_B, c, L = sp.symbols('nu T h k_B c L', positive=True, real=True)
dnu, dR = sp.symbols('dnu dR', positive=True, real=True)
n = sp.symbols('n', integer=True, nonnegative=True)

# ------------------------------
# Mode counting (per Hz per volume)
# ------------------------------
R = 2*L*nu/c
N_shell = sp.Rational(1,8) * 4*sp.pi*R**2 * dR     # 1/8 octant
N_all = sp.simplify(2 * N_shell)                    # ×2 polarizations
assert sp.simplify(N_all - sp.pi*R**2*dR) == 0
N_all_freq = sp.simplify(N_all.subs({R: 2*L*nu/c, dR: 2*L*dnu/c}))
rho_nu = sp.simplify(N_all_freq/(L**3 * dnu))
rho_nu_expected = sp.simplify(8*sp.pi*nu**2/c**3)
assert sp.simplify(rho_nu - rho_nu_expected) == 0

# ------------------------------
# Partition function and Bose–Einstein occupancy via q-series
# ------------------------------
x = sp.symbols('x', positive=True, real=True)                 # x = h nu / (k_B T)
q = sp.symbols('q', real=True)
S = sp.summation(q**n, (n,0, sp.oo))                          # = Piecewise(1/(1-q), |q|<1, ...)
S_n = sp.summation(n*q**n, (n,0, sp.oo))                      # = Piecewise(q/(1-q)**2, |q|<1, ...)
Z_q = S.args[0][0]                                            # 1/(1-q) branch
n_avg_q = sp.simplify(S_n.args[0][0]/Z_q)                     # q/(1-q)
Z_eval = sp.simplify(Z_q.subs(q, sp.exp(-x)).subs(x, h*nu/(k_B*T)))
n_avg_eval = sp.simplify(n_avg_q.subs(q, sp.exp(-x)).subs(x, h*nu/(k_B*T)))
assert sp.simplify(n_avg_eval - 1/(sp.exp(h*nu/(k_B*T)) - 1)) == 0

# Average energy per mode and Planck law u(nu,T)
E_avg = sp.simplify(h*nu * n_avg_eval)
u_planck = sp.simplify(rho_nu_expected * E_avg)
u_expected = sp.simplify((8*sp.pi*h*nu**3)/(c**3 * (sp.exp(h*nu/(k_B*T)) - 1)))
assert sp.simplify(u_planck - u_expected) == 0

# ------------------------------
# ω–ν consistency (use ħ = h/(2π))
# ------------------------------
omega, hbar = sp.symbols('omega hbar', positive=True, real=True)
u_omega = sp.simplify((omega**2)/(sp.pi**2*c**3) * (hbar*omega)/(sp.exp(hbar*omega/(k_B*T)) - 1))
check_omega_to_nu = sp.simplify(u_omega.subs({hbar: h/(2*sp.pi), omega: 2*sp.pi*nu}) * (2*sp.pi) - u_expected)
assert check_omega_to_nu == 0

# ------------------------------
# Rayleigh–Jeans (low-ν) and Wien (high-ν) limits
# ------------------------------
u_in_x = sp.simplify((8*sp.pi*(k_B*T)**3)/(c**3*h**2) * x**3/(sp.exp(x) - 1))
# RJ: need series to order >= 3 to capture x^2 term
u_RJ_series = sp.simplify(sp.series(u_in_x, x, 0, 3).removeO().subs(x, h*nu/(k_B*T)))
u_RJ_expected = sp.simplify(8*sp.pi*nu**2*k_B*T/c**3)
assert sp.simplify(u_RJ_series - u_RJ_expected) == 0
# Wien: ratio → 1 as ν→∞
ratio_Wien = sp.simplify(u_expected / ((8*sp.pi*h*nu**3)/(c**3) * sp.exp(-h*nu/(k_B*T))))
assert sp.limit(ratio_Wien, nu, sp.oo) == 1

# ------------------------------
# Stefan–Boltzmann: ∫_0^∞ u(ν,T) dν = a T^4, a = 8π^5 k_B^4 / (15 c^3 h^3)
# Use Γ–ζ identity: ∫_0^∞ x^{s-1}/(e^x - 1) dx = Γ(s) ζ(s), s=4 ⇒ π^4/15
# Also verify numerically to avoid unevaluated Integral
# ------------------------------
a_const = sp.simplify(8*sp.pi**5 * k_B**4 / (15 * c**3 * h**3))
I_closed = sp.gamma(4) * sp.zeta(4)                         # = π^4/15
I_numeric = sp.N(sp.Integral(sp.Symbol('xx')**3/(sp.exp(sp.Symbol('xx'))-1), (sp.Symbol('xx'), 0, sp.oo)))
assert abs(float(I_numeric - sp.N(I_closed))) < 1e-10
u_total_from_x = sp.simplify((8*sp.pi*(k_B*T)**4)/(c**3*h**3) * I_closed)
assert sp.simplify(u_total_from_x - a_const*T**4) == 0

# ------------------------------
# Wien's displacement (frequency form): maximize f(x)=x^3/(e^x-1)
# 3(1 - e^{-x}) - x = 0 ⇒ x_peak ≈ 2.821439372...
# ------------------------------
x_var = sp.symbols('x_var', positive=True)
eq_wien = sp.Eq(3*(1 - sp.exp(-x_var)) - x_var, 0)
x_peak = sp.nsolve(eq_wien, 3)     # good initial guess near 3
assert abs(float(x_peak) - 2.821439372) < 1e-6

print('All blackbody verifications passed ✔')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=blackbody_radiation.json](https://theoria-dataset.org/entries.html?entry=blackbody_radiation.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0